In [9]:
import torch
from torch import nn
import torch.nn.functional as F
# CenteredLayer 自定义层的作用：
# 这个层实现了数据中心化操作，即将输入张量的每个元素减去该张量的均值
# 前向传播的结果就是：输出 = 输入 - 输入的均值
# 这样可以使输出数据的均值为0，常用于数据预处理或正则化

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,X):
        return X-X.mean()  # 每个值减去均值，实现中心化

# 方法1：将CenteredLayer与Linear层组合使用
# 创建一个包含Linear层和CenteredLayer的网络
net = nn.Sequential(
    nn.Linear(5, 3),  # 输入5维，输出3维的线性层
    CenteredLayer()   # 对线性层的输出进行中心化
)

# 测试组合网络
input_tensor = torch.randn(2, 5, requires_grad=True)  # 批量大小为2，特征维度为5
output = net(input_tensor)
print("组合网络前向传播结果:", output)
print("输出的均值（应该接近0）:", output.mean())

# 方法2：直接对Linear层的输出使用CenteredLayer
linear_layer = nn.Linear(5, 3)
centered_layer = CenteredLayer()

# 先通过线性层，再通过中心化层
linear_output = linear_layer(input_tensor)
final_output = centered_layer(linear_output)
print("\n分步执行结果:", final_output)
print("分步执行输出的均值:", final_output.mean())

# 反向传播示例
loss = final_output.sum()
loss.backward()
print("\n输入张量的梯度形状:", input_tensor.grad.shape)

组合网络前向传播结果: tensor([[ 0.2678, -0.1506, -0.3099],
        [ 0.7303, -0.7609,  0.2234]], grad_fn=<SubBackward0>)
输出的均值（应该接近0）: tensor(0., grad_fn=<MeanBackward0>)

分步执行结果: tensor([[ 0.0189, -0.8486, -0.0878],
        [ 0.4596,  0.3599,  0.0981]], grad_fn=<SubBackward0>)
分步执行输出的均值: tensor(4.9671e-09, grad_fn=<MeanBackward0>)

输入张量的梯度形状: torch.Size([2, 5])


In [10]:
net=nn.Sequential(nn.Linear(8,128),CenteredLayer())
Y=net(torch.rand(4,8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

In [13]:
# MyLinear 自定义层的作用：
# 这个层实现了一个线性变换 + ReLU激活函数的组合
# 相当于 nn.Linear + nn.ReLU 的功能
# 前向传播过程：输入 -> 线性变换(矩阵乘法 + 偏置) -> ReLU激活

class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(in_units,units))  # 权重参数
        self.bias=nn.Parameter(torch.randn(units,))            # 偏置参数
    
    def forward(self,X):
        # 步骤1：线性变换 (相当于nn.Linear的功能)
        linear=torch.matmul(X,self.weight.data)+self.bias.data
        # 步骤2：ReLU激活函数 (相当于nn.ReLU的功能)
        return F.relu(linear)

# 创建自定义层实例
linear=MyLinear(5,3)  # 输入5维，输出3维
linear.weight         # 查看权重参数

Parameter containing:
tensor([[ 0.0597, -0.7242, -0.0976],
        [-0.8326, -1.6497, -1.8903],
        [ 1.1323, -1.3238,  0.4103],
        [ 0.9623, -1.3757, -0.0989],
        [ 0.7987,  0.1529, -0.3622]], requires_grad=True)

In [16]:
# 测试自定义层的前向传播
input_data = torch.rand(2,5)  # 输入：2个样本，每个样本5个特征
output = linear(input_data)   # 输出：2个样本，每个样本3个特征

print(f"输入形状: {input_data.shape}")  # torch.Size([2, 5])
print(f"权重形状: {linear.weight.shape}")  # torch.Size([5, 3])
print(f"输出形状: {output.shape}")  # torch.Size([2, 3])
print(f"输出结果:\n{output}")

# 矩阵运算解释：
# 输入 (2×5) × 权重 (5×3) = 输出 (2×3)
# 这是标准的矩阵乘法规则：(m×n) × (n×p) = (m×p)

输入形状: torch.Size([2, 5])
权重形状: torch.Size([5, 3])
输出形状: torch.Size([2, 3])
输出结果:
tensor([[0.3846, 0.0000, 1.0422],
        [0.6881, 0.0000, 0.6264]])


In [40]:
net=nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[1.7160],
        [6.4937]])